In [1]:
import os
import xarray as xr
import pandas as pd
import numpy as np
from tensorflow import keras
import random
from tensorflow.keras import layers, models, Input
import tensorflow as tf

folder = "../../data/model_data/mini_data_set"
full_paths = [os.path.join(folder, f) for f in os.listdir(folder)]
random.shuffle(full_paths)

sensor_df = pd.read_csv("../../data/sensor_data/final_sensor_withgrid.csv")
sensor_df = sensor_df[['lat','lon']]
sensor_df = sensor_df.drop_duplicates().reset_index(drop=True)

# Shuffle the indices
sensor_df = sensor_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Assign groups evenly
sensor_df['cv_group'] = sensor_df.index % 5 + 1  # Groups: 0 to 4

2025-08-01 19:00:53.479854: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-01 19:00:54.407090: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/lib/x86_64-linux-gnu/:/opt/conda/lib
2025-08-01 19:00:54.407204: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local

In [2]:
class ncDataLoader(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, img_paths, drop_sensor_locs_df):
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_paths = img_paths
        self.drop_sensor_locs_df = drop_sensor_locs_df

    def __len__(self):
        return len(self.img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.img_paths[i : i + self.batch_size]

        x = np.zeros((self.batch_size,) + self.img_size + (22,), dtype="float32")
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="float32")

        for j, path in enumerate(batch_input_img_paths):
            # print(path)
            ds = xr.open_dataset(path)
            input_ds = ds.copy()
            input_ds = input_ds.drop_vars("y_sensor_no2")
            input_array = input_ds.to_array().values
            x[j] = np.transpose(input_array, (1, 2, 0))
            
            target_ds = ds['y_sensor_no2']
            
            # Loop over rows to mask each (lat, lon) pair
            for _, row in self.drop_sensor_locs_df.iterrows():
                target_ds.loc[dict(lat=row['lat'], lon=row['lon'])] = np.nan
            
            # Replace NaNs with -999
            target_ds = target_ds.values
            # target_ds = np.where(np.isnan(target_ds), -999, target_ds)
            y[j, :, :, 0] = target_ds
 
        return x, y



def build_baseline_model(
    input_shape=(479, 1059, 22)
):
    """
    Build a convolution-free, per-pixel linear regression model.
    Each pixel's 22 input features are combined linearly to produce 1 output value.
    
    Args:
        input_shape (tuple): Input shape (H, W, C)

    Returns:
        Keras Model
    """
    inputs = Input(shape=input_shape)
    x = layers.BatchNormalization()(inputs)

    # Per-pixel linear regression (no hidden layer)
    outputs = layers.Dense(1, activation='linear', use_bias=True)(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    return model


def masked_mse(y_true, y_pred):
    # Create mask: True where y_true is not NaN
    mask = tf.math.logical_not(tf.math.is_nan(y_true))
    # tf.print("Valid pixels:", tf.reduce_sum(tf.cast(mask, tf.int32)))

    # Apply mask to both y_true and y_pred
    y_true_masked = tf.boolean_mask(y_true, mask)
    y_pred_masked = tf.boolean_mask(y_pred, mask)
    
    # Compute MSE only on valid values
    diff_squared = tf.square(y_true_masked - y_pred_masked)
    
    # Avoid division by zero
    denom = tf.cast(tf.size(diff_squared), tf.float32)
    denom = tf.maximum(denom, 1e-6)

    return tf.reduce_sum(diff_squared) / denom


In [ ]:
################################################
##      Cross Validation and Save Results     ##
################################################

# Parameters
batch_size = 4
img_size = (479, 1059)
img_paths = full_paths
kernel_sizes_param = [3, 5, 3]
CNN_filters_param = 64
activation_param = 'relu'
epochs_param = 4

cv_results = []

for cv_group in range(1, 6):  # cv_group 1 to 5
    print(f"Running CV group {cv_group}/5")
    
    drop_locs_train_set = sensor_df[sensor_df['cv_group'] == cv_group][['lat', 'lon']]
    drop_locs_val_set = sensor_df[sensor_df['cv_group'] != cv_group][['lat', 'lon']]

    train_loader = ncDataLoader(
        batch_size=batch_size,
        img_size=img_size,
        img_paths=full_paths,
        drop_sensor_locs_df=drop_locs_train_set
    )

    val_loader = ncDataLoader(
        batch_size=batch_size,
        img_size=img_size,
        img_paths=full_paths,
        drop_sensor_locs_df=drop_locs_val_set
    )

    # Reset model for each fold
    model = build_baseline_model(input_shape=(479, 1059, 22))
    model.compile(optimizer='adam', loss=masked_mse)

    model.fit(
        train_loader,
        validation_data=val_loader,
        epochs=epochs_param,
        verbose=1
    )

    # Predict and collect results
    for i, (batch_x, batch_y_true) in enumerate(val_loader):
        batch_y_pred = model.predict(batch_x)

        for j in range(len(batch_y_true)):
            y_true = batch_y_true[j]
            y_pred = batch_y_pred[j]

            mask = ~np.isnan(y_true)
            rows, cols, _ = np.where(mask)

            true_vals = y_true[rows, cols]
            pred_vals = y_pred[rows, cols]

            # Optional: use corresponding file name for traceability
            file_idx = i * batch_size + j
            file_name = os.path.basename(full_paths[file_idx]) if file_idx < len(full_paths) else None

            for r, c, t, p in zip(rows, cols, true_vals, pred_vals):
                cv_results.append({
                    'cv_group': cv_group,
                    'file': file_name,
                    'row': r,
                    'col': c,
                    'true': t[0],
                    'predicted': p[0],
                    'lat': r * 0.01 + 28.605,
                    'lon': c * 0.01 -98.895
                })

df_cv = pd.DataFrame(cv_results)

Running CV group 1/5


2025-08-01 19:01:05.973845: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/lib/x86_64-linux-gnu/:/opt/conda/lib
2025-08-01 19:01:05.973901: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2025-08-01 19:01:05.973930: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sat-processing): /proc/driver/nvidia/version does not exist
2025-08-01 19:01:05.974175: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other

Epoch 1/4
12/12 [==============================] - 153s 13s/step - loss: 15.9654 - val_loss: 2606.6765
Epoch 2/4
12/12 [==============================] - 151s 13s/step - loss: 9.6384 - val_loss: 1172.1293
Epoch 3/4
12/12 [==============================] - 151s 13s/step - loss: 5.8383 - val_loss: 381.0619
Epoch 4/4
1/1 [==============================] - 2s 2s/step
Running CV group 2/5
Epoch 1/4
12/12 [==============================] - 153s 13s/step - loss: 12.8233 - val_loss: 5819.6055
Epoch 2/4
12/12 [==============================] - 152s 13s/step - loss: 6.7775 - val_loss: 3158.4729
Epoch 3/4
12/12 [==============================] - 154s 13s/step - loss: 4.6622 - val_loss: 6379.2046
Epoch 4/4
1/1 [==============================] - 2s 2s/step
Running CV group 3/5
Epoch 1/4
12/12 [==============================] - 152s 13s/step - loss: 12.8773 - val_loss: 5479.7983
Epoch 2/4
12/12 [==============================] - 150s 13s/step - loss: 7.9126 - val_loss: 2854.2617
Epoch 3/4
12/12 [===

In [8]:
df_cv.to_csv('test_model_results.csv')

In [10]:
sensor_df.to_csv('cv_groups_and_lat_lon.csv')